In [195]:
import pandas as pd

In [196]:
# To predict Sale price
df = pd.read_csv('House price adv regression\\train.csv')
df.head()


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [197]:
to_Label_encode = ['MSZoning','LotShape','LandContour','Utilities','LotConfig','LandSlope','BldgType','HouseStyle','ExterQual','ExterCond','BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2','HeatingQC','CentralAir','KitchenQual','FireplaceQu','GarageQual','GarageCond', 'PavedDrive','Fence']
to_OHE = ['Street','Alley','Neighborhood','Condition1','Condition2','RoofStyle','RoofMatl','Exterior1st','Exterior2nd','Foundation','Heating','Electrical','Functional','GarageType','SaleType','SaleCondition','GarageFinish']

# to remove : MasVnrType','PoolQC','MiscFeature


In [198]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [199]:
df['Electrical'].mode()

0    SBrkr
Name: Electrical, dtype: object

In [200]:
def clean(curr_data):
    # Correct version of the operations
    curr_data['Alley'] = curr_data['Alley'].fillna(0)
    curr_data['LotFrontage'] = curr_data['LotFrontage'].fillna(curr_data['LotFrontage'].median())
    curr_data = curr_data.drop(columns=['MasVnrType','PoolQC','MiscFeature','Id'])
    curr_data['MasVnrArea'] = curr_data['MasVnrArea'].fillna(curr_data['MasVnrArea'].median())
    curr_data['Electrical'] = curr_data['Electrical'].fillna('SBrkr')
    curr_data['GarageYrBlt'] = curr_data['GarageYrBlt'].fillna(curr_data['GarageYrBlt'].median())
    curr_data['BsmtQual'] = curr_data['BsmtQual'].fillna('0')
    curr_data['BsmtCond'] = curr_data['BsmtCond'].fillna('0')
    curr_data['BsmtExposure'] = curr_data['BsmtExposure'].fillna('0')
    curr_data['BsmtFinType1'] = curr_data['BsmtFinType1'].fillna('0')
    curr_data['BsmtFinType2'] = curr_data['BsmtFinType2'].fillna('0')
    curr_data['FireplaceQu'] = curr_data['FireplaceQu'].fillna('0')
    curr_data['GarageType'] = curr_data['GarageType'].fillna('0')
    curr_data['GarageFinish'] = curr_data['GarageFinish'].fillna('0')
    curr_data['GarageQual'] = curr_data['GarageQual'].fillna('0')
    curr_data['GarageCond'] = curr_data['GarageCond'].fillna('0')
    curr_data['Fence'] = curr_data['Fence'].fillna('0')
    from sklearn.preprocessing import LabelEncoder
    le = LabelEncoder()
    for col in to_Label_encode:
        curr_data[col] = le.fit_transform(curr_data[col])

    one_hot = pd.get_dummies(curr_data[to_OHE]).astype(int)
    curr_data1 = curr_data.drop(columns=to_OHE)
    curr_data_encoded = pd.concat([curr_data1, one_hot], axis=1)
    return curr_data_encoded


In [201]:

x_pre = df.drop(columns=['SalePrice'])
x_pre.shape

(1460, 80)

In [202]:
x = clean(x_pre)
y = df['SalePrice']

In [203]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(x,y)

LinearRegression()

In [204]:
len(x.columns)

201

In [214]:
x_test = pd.read_csv('House price adv regression\\test.csv')
idCol = x_test['Id']
x_test.shape


(1459, 80)

In [218]:
df_combined = pd.concat([x_test,df.drop(columns=['SalePrice'])],axis=0)
df_combined = clean(df_combined)

x_test1 = df_combined.head(1459)
columns_with_nan = x_test1.columns[x_test1.isna().any()].tolist()
# x_test1['LotFrontage'].info()
Nan_list = ['BsmtFinSF1','BsmtFinSF2','BsmtUnfSF','GarageCars','TotalBsmtSF','BsmtFullBath','BsmtHalfBath','GarageArea']

for col in Nan_list:
    x_test1[col] = x_test1[col].fillna(x_test1[col].median())

C:\Users\samiu\AppData\Local\Temp\ipykernel_1156\2274673807.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_test1[col] = x_test1[col].fillna(x_test1[col].median())
C:\Users\samiu\AppData\Local\Temp\ipykernel_1156\2274673807.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_test1[col] = x_test1[col].fillna(x_test1[col].median())
C:\Users\samiu\AppData\Local\Temp\ipykernel_1156\2274673807.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .

In [219]:
y_pred = lr.predict(x_test1)

In [220]:
len(y_pred)

1459

In [221]:
df_submission = pd.DataFrame(y_pred)
df_submission['Id'] = idCol
df_submission.columns = ['SalePrice', 'Id']
df_submission = df_submission.reindex(columns=["Id","SalePrice"])

df_submission.to_csv("AdvLR.csv",index=False)